# Data Download

In [1]:
from fredapi import Fred
import pandas as pd
import numpy as np
import statsmodels.api as sm

fred = Fred(api_key='3cca2c46ad523e6d1bb9269895f879fc')

billion = 1000000000
thousand = 1000
 
# Downloading the data from FRED
gdp = fred.get_series('GDPC96')*billion # Real GDP, Billions of Chained 2009 Dollars, Seasonally Adjusted Annual Rate
cons = fred.get_series('PCECC96')*billion # Real Personal Consumption Expenditures, Billions of Chained 2009 Dollars, Seasonally Adjusted Annual Rate
inv = fred.get_series('PNFI')*billion # Private Nonresidential Fixed Investment, Billions of Dollars, Seasonally Adjusted Annual Rate

pop = fred.get_series('B230RC0Q173SBEA')*thousand # Population (midperiod), Thousands, Seasonally Adjusted Annual Rate

hours_index = fred.get_series("HOANBS") # Nonfarm Business Sector: Hours of All Persons, Index 2009=100, Seasonally Adjusted
average_weekly_hours = fred.get_series("AWHAETP") # Average Weekly Hours of All Employees: Total Private, Monthly

real_hourly_comp_index = fred.get_series("COMPRNFB")# Nonfarm Business Sector: Real Compensation Per Hour, Index 2009=100, Seasonally Adjusted
avg_hourly_comp = fred.get_series("CES0500000003") # Average Hourly Earnings of All Employees: Total Private Dollars per Hour, Seasonally Adjusted

i = fred.get_series('TB3MS') # 3-Month Treasury Bill: Secondary Market Rate, Monthly
cpi = fred.get_series('CPIAUCSL') # Consumer Price Index for All Urban Consumers: All Items, Index 1982-1984=100, Seasonally Adjusted

capital_stock = fred.get_series('K1NTOTL1ES000')*billion # Current-Cost Net Stock of Fixed Assets: Private: Nonresidential, Billions of Dollars, Seasonally Adjusted Annual Rate

# Importing the simulated data
simulated_data = pd.read_csv('simulated_data.csv', names=["Y","C","I","N","Y/N","w","r", "A"])

/Users/QBatista/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Data Processing

In [8]:
def data_processor(data, operation, freq):
    df = pd.DataFrame(np.array(data), index=pd.PeriodIndex(data.index, freq=freq), columns =[str(data)])
    if operation == "mean":
        df = df.groupby(pd.PeriodIndex(data.index, freq=freq), axis=0).mean().to_timestamp().loc["1947-01-01":"2017-01-01"]
    if operation == "sum":
        df = df.groupby(pd.PeriodIndex(data.index, freq=freq), axis=0).sum().to_timestamp().loc["1947-01-01":"2017-01-01"]
    return pd.Series(df[str(data)])

def table_replicator(data, detrending_method):
    if detrending_method == "hp_filter":
        data, trend = sm.tsa.filters.hpfilter(data, lamb=1600)
    if detrending_method == "first_difference":
        data = data.diff(periods=1, axis=0).fillna(0)
    data = data*100
    first_order_autocorr = data.apply(lambda col: col.autocorr(lag=1), axis=0)
    corr = data.corr()["Y"]
    df = pd.concat([np.std(data), np.std(data)/np.std(data)["Y"], first_order_autocorr, corr], axis=1).round(decimals=2)
    df.columns = ["Standard Deviation", "Relative Standard Deviation", "First Order Auto-correlation", "Contemporaneous Correlation with Output"]
    return df 

# Processing the interest rate series
i_Q = data_processor(i, "mean", "Q")

# Processing the cpi series
cpi_Q = data_processor(cpi, "mean", "Q").pct_change().fillna(0)

# Processing the average weekly hours time series
hours_2009 = float(data_processor(average_weekly_hours["2009-01-01":"2009-12-31"], "mean", "A")*52/4)

# Processing the average hourly earnings time series
avg_hourly_comp_2009 = float(data_processor(avg_hourly_comp["2009-01-01":"2009-12-31"], "mean", "A")*52/4)

# Quarterly Hours = Hours Index(index 2009=100) * Average Quarterly Hours (in 2009) / 100
q_hours = hours_index / 100 * hours_2009 

# Real Hourly Compensation Index(index 2009=100) * Average Quarterly Hours (in 2009) / 100
w = real_hourly_comp_index/100 * avg_hourly_comp_2009

# Real Interest Rate = Nominal Interest Rate - Inflation
r = i_Q/100 - cpi_Q

# Interpolating the capital stock time series
df = pd.concat([inv["1947-01-01":"2015-01-01"],capital_stock["1947-01-01":"2015-01-01"]], axis=1)
df.columns = ["inv", "capital stock"]
K = df["capital stock"].interpolate(method="quadratic")

# log(TFP) = log(Y) - alpha*log(K) - (1-alpha)*log(N)
alpha = 0.35
TFP = np.log(gdp)-alpha*np.log(K)-(1-alpha)*np.log(q_hours)

# Creating the data frame
data = pd.concat([gdp/pop, cons/pop, inv["1947-01-01":"2017-01-01"]/pop ,q_hours, (gdp/pop)/q_hours, w, r, TFP], axis=1)["1947-01-01":"2015-01-01"]
data.columns = ["Y","C","I","N","Y/N","w","r", "A"]

# Taking logs
data[["Y","C","I","N","Y/N","w"]] = np.log(data[["Y","C","I","N", "Y/N","w"]])

# Results

In [9]:
### Note: For comparing with the paper results, use data["1947-01-01":"1996-12-31"] instead of data

# Replication of Table 1 with HP filter
table_1_hp_filter = table_replicator(data, "hp_filter")
table_1_hp_filter

,Standard Deviation,Relative Standard Deviation,First Order Auto-correlation,Contemporaneous Correlation with Output
Y,1.65,1.00,0.85,1.00
C,1.28,0.78,0.82,0.78
I,4.78,2.90,0.88,0.68
N,1.92,1.16,0.91,0.87
Y/N,0.94,0.57,0.76,-0.03
w,0.96,0.58,0.68,0.15
r,1.04,0.63,0.71,0.24
A,1.08,0.66,0.82,0.31


In [10]:
# Replication of Table 1 with first order difference
table_1_diff = table_replicator(data, "first_difference")
table_1_diff

,Standard Deviation,Relative Standard Deviation,First Order Auto-correlation,Contemporaneous Correlation with Output
Y,0.95,1.00,0.37,1.00
C,0.82,0.87,0.09,0.59
I,2.52,2.65,0.50,0.56
N,0.90,0.95,0.64,0.74
Y/N,0.67,0.71,0.06,0.43
w,0.81,0.85,-0.06,0.12
r,0.81,0.85,0.00,0.21
A,0.73,0.77,0.23,0.62


In [13]:
# Replication of Table 3 with HP filter 
table_3_hp_filter = table_replicator(simulated_data, "hp_filter")
table_3_hp_filter

,Standard Deviation,Relative Standard Deviation,First Order Auto-correlation,Contemporaneous Correlation with Output
Y,1.13,1.00,0.69,1.00
C,0.37,0.33,0.78,0.92
I,2.19,1.95,0.68,1.00
N,0.69,0.61,0.67,0.98
Y/N,0.47,0.42,0.74,0.96
w,0.47,0.42,0.74,0.96
r,0.19,0.17,0.67,0.98
A,0.85,0.76,0.68,0.99


In [12]:
# Replication of Table 3 with first order difference
table_3_diff = table_replicator(simulated_data, "first_difference")
table_3_diff

,Standard Deviation,Relative Standard Deviation,First Order Auto-correlation,Contemporaneous Correlation with Output
Y,0.93,1.00,-0.01,1.00
C,0.30,0.32,0.29,0.91
I,1.81,1.95,-0.03,1.00
N,0.57,0.61,-0.05,0.98
Y/N,0.38,0.41,0.16,0.96
w,0.38,0.41,0.16,0.96
r,0.16,0.17,-0.05,0.98
A,0.71,0.76,-0.04,0.99
